In [15]:
import sys
from pathlib import Path

# Añade src al path
sys.path.append(str(Path().resolve().parent / "src"))


In [16]:
# para actualizar cambios del .py en el IPython notebook
%reload_ext autoreload
%autoreload 2

In [17]:
from data import load_raw_data

# Descargamos el archivo de datos crudos
# para Enero de 2024
# Cargamos el archivo de datos crudos
# Validamos el archivo con las fechas
df = load_raw_data(2024, [1])

# Mostramos estadísticos
df.describe()

Archivo 2024-01 ya está disponible localmente


,pickup_datetime,pickup_location_id
count,2964606,2.964606e+06
mean,2024-01-17 01:02:08.093335,1.660179e+02
min,2024-01-01 00:00:00,1.000000e+00
25%,2024-01-09 15:59:24,1.320000e+02
50%,2024-01-17 10:45:42,1.620000e+02
75%,2024-01-24 18:23:53,2.340000e+02
max,2024-01-31 23:59:55,2.650000e+02
std,NaN,6.362399e+01


In [18]:
from data import transform_to_time_series

# Transformamos el dataframe a una serie temporal
ts_data = transform_to_time_series(df)
ts_data

/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:146: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:111: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
Rellenando huecos por localización: 100%|██████████| 260/260 [00:00<00:00, 1315.73it/s]


,pickup_hour,pickup_location_id,rides
0,2024-01-01 00:00:00,4,25
1,2024-01-01 01:00:00,4,29
2,2024-01-01 02:00:00,4,34
3,2024-01-01 03:00:00,4,31
4,2024-01-01 04:00:00,4,32
...,...,...,...
193435,2024-01-31 19:00:00,245,0
193436,2024-01-31 20:00:00,245,0
193437,2024-01-31 21:00:00,245,0
193438,2024-01-31 22:00:00,245,0


In [12]:
ts_data[ts_data.pickup_location_id == 138]

,pickup_hour,pickup_location_id,rides
55800,2024-01-01 00:00:00,138,59
55801,2024-01-01 01:00:00,138,4
55802,2024-01-01 02:00:00,138,0
55803,2024-01-01 03:00:00,138,0
55804,2024-01-01 04:00:00,138,0
...,...,...,...
56539,2024-01-31 19:00:00,138,200
56540,2024-01-31 20:00:00,138,205
56541,2024-01-31 21:00:00,138,137
56542,2024-01-31 22:00:00,138,222


In [24]:
from data import transform_to_features_and_target

# Transformamos la serie temporal a un dataframe de características y objetivo
location_id = 138
n_lags = 24*14 # 1 días

X, y, df_location = transform_to_features_and_target(ts_data, location_id, n_lags)
df_location


/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:178: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:178: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'rides_previous_{lag}_hour'] = df['rides'].shift(lag)
/Users/cdonairem/Documents/Workspace/proyecto_mlops_prueba_poetry/src/data.py:178: PerformanceWarning: DataFrame is highly fragmented.  This is usually the resu

,pickup_hour,pickup_location_id,rides_previous_336_hour,rides_previous_335_hour,rides_previous_334_hour,rides_previous_333_hour,rides_previous_332_hour,rides_previous_331_hour,rides_previous_330_hour,rides_previous_329_hour,...,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,target
0,2024-01-15 00:00:00,138,59.0,4.0,0.0,0.0,0.0,1.0,1.0,8.0,...,154.0,148.0,217.0,188.0,269.0,160.0,178.0,223.0,361.0,240
1,2024-01-15 01:00:00,138,4.0,0.0,0.0,0.0,1.0,1.0,8.0,64.0,...,148.0,217.0,188.0,269.0,160.0,178.0,223.0,361.0,240.0,15
2,2024-01-15 02:00:00,138,0.0,0.0,0.0,1.0,1.0,8.0,64.0,139.0,...,217.0,188.0,269.0,160.0,178.0,223.0,361.0,240.0,15.0,8
3,2024-01-15 03:00:00,138,0.0,0.0,1.0,1.0,8.0,64.0,139.0,199.0,...,188.0,269.0,160.0,178.0,223.0,361.0,240.0,15.0,8.0,2
4,2024-01-15 04:00:00,138,0.0,1.0,1.0,8.0,64.0,139.0,199.0,138.0,...,269.0,160.0,178.0,223.0,361.0,240.0,15.0,8.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,2024-01-31 19:00:00,138,223.0,206.0,125.0,225.0,181.0,49.0,2.0,1.0,...,152.0,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200
404,2024-01-31 20:00:00,138,206.0,125.0,225.0,181.0,49.0,2.0,1.0,0.0,...,132.0,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205
405,2024-01-31 21:00:00,138,125.0,225.0,181.0,49.0,2.0,1.0,0.0,0.0,...,157.0,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137
406,2024-01-31 22:00:00,138,225.0,181.0,49.0,2.0,1.0,0.0,0.0,2.0,...,143.0,200.0,170.0,145.0,142.0,197.0,200.0,205.0,137.0,222


In [25]:
# guardamos datos
from paths import TRANSFORMED_DATA_DIR

df_location.to_parquet(TRANSFORMED_DATA_DIR / 'tabular_data_14d.parquet')